# MultiProcessing - Pool

## 기능요구사항
### 작업 정의:

각 작업에 이름(예: 'A', 'B' 등)과 기간(초를 나타내는 정수)을 지정하여 작업 목록을 정의합니다.

### 워커 풀 설정:

지정된 수의 작업자(이 경우 2개)로 풀을 초기화합니다.

### 작업 실행:

작업 목록에 있는 각 작업에 대해 작업 이름과 기간을 나타내는 대기 메시지를 인쇄합니다.

time.sleep을 사용하여 지정된 시간 동안 잠을 자면서 작업을 시뮬레이션합니다.

작업을 완료한 후, 작업이 완료되었음을 나타내는 메시지를 인쇄합니다.

### 동시성:

워커 풀(Pool)을 사용하여 작업이 동시에 실행되는지 확인합니다.

## 프로그래밍관점
Python의 multiprocessing 모듈을 사용하세요.

각 작업이 이름과 기간으로 구성된 작업 목록을 정의합니다.

multiprocessing.Pool을 사용하여 여러 작업을 동시에 실행합니다.

대기 메시지를 출력한 다음 지정된 기간 동안 대기하여 각 작업을 시뮬레이션하는 함수(work_log)를 정의합니다.

Pool의 map 함수를 사용하여 work_log 함수를 작업 목록에 매핑합니다.

In [3]:
import multiprocessing
import time

In [6]:
tasks = { # task_name : duration(sec)
    'A' : 5,
    'B' : 2,
    'C' : 1,
    'D' : 3
} 

In [7]:
def work_log(name:str, wait:int):
    print(f"프로세스 {name}가 {wait}초간 대기")
    time.sleep(wait)
    print(f"프로세스 {name}가 완료됨.")

In [10]:
work_log('A', 2)

프로세스 A가 2초간 대기
프로세스 A가 완료됨.


In [11]:
with multiprocessing.Pool(processes=2) as pool:
    pool.map(work_log, [(tasks.keys, tasks.values)])

Process SpawnPoolWorker-7:
Traceback (most recent call last):
  File "/Users/admin/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/admin/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/admin/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/admin/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'work_log' on <module '__main__' (built-in)>


KeyboardInterrupt: 

In [13]:
from multiprocessing import Process, Queue, Lock, current_process
import queue
import time
    
def pull_task(pull_q:Queue, push_q:Queue, lock):
    while True:
        
        try:
            with lock:
                num = pull_q.get_nowait()
        
        except queue.Empty as e:
            print(f"Queue is empty: {current_process().name}", e)
            break
        
        else:
            time.sleep(0.5)
            push_q.put(msg := f"작업 번호 {num}은 {current_process().name}에 의해 수행됨")
            print(f"Push {num} by {current_process().name}")
    
    


tasks_to_accomplish = Queue()
tasks_that_are_done = Queue()

lock = Lock()

for i in range(10):
    print(f"작업 번호 {i}")
    tasks_to_accomplish.put(i)
    
            
procs = [Process(target=pull_task, args=(tasks_to_accomplish, tasks_that_are_done, lock))for _ in range(4)]

for proc in procs:
    proc.start()
    
for proc in procs:
    proc.join()
    
while not tasks_that_are_done.empty():
    print(tasks_that_are_done.get())

tasks_to_accomplish.close()
tasks_to_accomplish.join_thread()

tasks_that_are_done.close()
tasks_that_are_done.join_thread()


작업 번호 0
작업 번호 1
작업 번호 2
작업 번호 3
작업 번호 4
작업 번호 5
작업 번호 6
작업 번호 7
작업 번호 8
작업 번호 9


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/admin/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'pull_task' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/admin/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/.pyenv/versions/3.11.11/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^